<a href="https://colab.research.google.com/github/Valentinafoschi/text2relax/blob/main/rumori_bianchi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

questo progetto ha come obiettivo quello di generare un audio + video rilassanti a partire da una descrizione testuale.

testo --> paesaggi sonori + visivi

In [1]:
%pip install --upgrade --quiet numpy==1.24.4 scipy==1.11.1


In [2]:
%pip install --upgrade --quiet librosa==0.10.1 soundfile==0.12.1 moviepy==1.0.3


In [3]:
%pip install --upgrade --quiet torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 \
  open-clip-torch==2.24.0


facciamo un check per vedere se funziona tutto (da eliminare in fase di consegna del progetto)

In [4]:
import numpy, scipy, librosa, torch, moviepy, soundfile
print("NumPy", numpy.__version__)
print("SciPy", scipy.__version__)
print("librosa", librosa.__version__)
print("PyTorch", torch.__version__)


NumPy 1.24.4
SciPy 1.11.1
librosa 0.10.1
PyTorch 2.2.2+cu121


di seguito importiamo tutte le librerie necessarie per il progetto


In [5]:
import os, json, random, math, colorsys, re
from typing import Dict, Any, List, Tuple

import numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import librosa, soundfile as sf
from scipy.signal import butter, lfilter, fftconvolve, welch
from moviepy.editor import VideoClip, AudioFileClip

import open_clip

# CREAZIONE CARTELLE BASE PROGETTO
for d in ["data","results","checkpoints"]:
    os.makedirs(d, exist_ok=True)

# seed + device
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)



'cpu'

scegliamo le caratteristiche visive che il nostro modello deve generare e i tipi di audio che dovrà poi prevedere.
pattern scelti:
- **electric** --> lampi o scariche che attraversano lo schermo
***snow** ---> fiocchi di neve che cadono
***waves** --> onde sinusoidali lente
***ripple** --> onde circolari concentriche tipo goccia in acqua
***rotating shapes** --> cerchi, quadrati o triangoli che ruotano lentamente



In [6]:
NOISE_TYPES = ["white", "pink", "brown", "blue"] #audio
PATTERNS = ["electric", "snow", "waves", "ripple", "rotatingShapes"] #video

RANGES = {
    # AUDIO
    "duration_sec": (30, 90),
    "lp_cutoff": (300, 8000),   # filtro low-pass
    "hp_cutoff": (20, 2000),    # filtro high-pass
    "reverb_mix": (0.0, 0.6),   # quantità di riverbero
    "fade_in_sec": (0.0, 5.0),  # durata del fade-in
    "fade_out_sec": (0.0, 5.0), # durata del fade-out
    "rain_level": (0.0, 1.0),   # intensità del pioggia
    "waves_level": (0.0, 1.0),  # intensità delle onde
    "ripple_level": (0.0, 1.0), # intensità delle goccie

    # VIDEO
    "hue_main": (0.0, 1.0),     # tonalità colore HSV
    "sat": (0.2, 1.0),          # saturazione colore HSV
    "value": (0.2, 1.0),        # luminosità colore HSV
    "motion_speed": (0.05, 0.6),    # velocità movimento
    "motion_amplitude": (0.1, 1.0), # ampiezza movimento
    "scene_brightness": (0.2, 1.0), # luminosità globale
    "sync_to_audio": (0.0, 1.0),    # sincronizzazione con audio

}


# funzione per normalizzare un parametro
def norm_param (name, x):
  lo, hi = RANGES[name]
  return (x - lo) / (hi - lo + 1e-9)


# funzione per denormalizzare un parametro
def denorm_param (name, x01):
  lo, hi = RANGES[name]
  return float(lo + x01 * (hi - lo))

nella cella seguente ci occupiamo del renderer audio, cioè la parte che a partire dai parametri audio previsti dal modello genera il file .wav vero e proprio.

In [7]:
SR = 22050 # quante vote viene campionato il segnale audio

# filtraggio frequenza
def _butter_filter (x, cutoff, btype):
  if cutoff <= 0:
    return x
  nyq = 0.5 * SR
  cutoff = min(cutoff, nyq-100)
  b, a = butter(4, cutoff / nyq, btype=btype)
  return lfilter(b, a, x)

# generazione rumore base
def _gen_noise (kind, n):
  if kind == "white": #frequenze uguali
    x = np.random.randn(n)
  elif kind=="pink": # più energia sulle basse frequenze
    x = _butter_filter(np.random.randn(n), 500, 'low')
  elif kind=="brown": # ancora più basse frequenze
    x = np.cumsum(np.random.randn(n)); x /= (np.abs(x).max()+1e-9)
  elif kind=="blue": # frequenze più alte
    x = _butter_filter(np.random.randn(n), 3000, 'high')
  return x.astype(np.float32)

# genera un'onda alla frequenza
def _sine(freq, n): t = np.arange(n)/SR; return np.sin(2*np.pi*freq*t).astype(np.float32)

#simula le gocce di pioggia
def _rain(n, level):
    if level<=1e-3: return np.zeros(n, np.float32)
    y = np.zeros(n, np.float32); drops = int(level*3*n/SR)
    for _ in range(drops):
        i = np.random.randint(0, n-200)
        y[i:i+200] += (np.hanning(200)*np.random.uniform(0.3,1.0)).astype(np.float32)
    return _butter_filter(y, 8000, 'low')

# genera le onde del mare
def _waves(n, level):
    if level<=1e-3: return np.zeros(n, np.float32)
    base = _sine(np.random.uniform(0.1,0.25), n)*0.3
    mod  = _sine(np.random.uniform(0.01,0.05), n)*0.7 + 0.7
    return (base*mod*level).astype(np.float32)



# FUNZIONE PRINCIPALE DI RENDERING AUDIO

def render_audio(params: Dict[str, Any], out_wav: str):
    n = int(params["duration_sec"]*SR)
    x = _gen_noise(params["noise_type"], n)
    if params["lp_cutoff"]>0: x = _butter_filter(x, params["lp_cutoff"], 'low')
    if params["hp_cutoff"]>0: x = _butter_filter(x, params["hp_cutoff"], 'high')
    x = 0.7*x + 0.2*_rain(n, params["rain_level"]) + 0.3*_waves(n, params["waves_level"])
    if params["reverb_mix"]>0:
        ir = np.exp(-np.linspace(0,1.0,int(0.12*SR))).astype(np.float32)
        wet = fftconvolve(x, ir)[:n]
        x = (1-params["reverb_mix"])*x + params["reverb_mix"]*wet
    fi = int(params["fade_in_sec"]*SR); fo = int(params["fade_out_sec"]*SR)
    if fi>0: x[:fi] *= np.linspace(0,1,fi)
    if fo>0: x[-fo:] *= np.linspace(1,0,fo)
    x /= (np.abs(x).max()+1e-9)
    sf.write(out_wav, x, SR); return out_wav

ora creiamo l'animazione in base ai parametri e la sincronizziamo con l'audio

In [16]:
from moviepy.editor import VideoClip
import numpy as np, colorsys

def _hsv_to_rgb01_scalar(h, s, v):
    r,g,b = colorsys.hsv_to_rgb(h % 1.0, np.clip(s,0,1), np.clip(v,0,1))
    return np.array([r, g, b], dtype=np.float32)

def render_video_safe(params: dict, out_mp4: str, fps=10, size=(320,180)):
    # Parametri
    duration = float(params.get("duration_sec", 5))
    hue  = float(params.get("hue_main", 0.55))
    sat  = float(params.get("sat", 0.9))
    val0 = float(params.get("value", 0.7))
    speed = float(params.get("motion_speed", 0.12))
    amp   = float(params.get("motion_amplitude", 0.5))
    pattern = params.get("pattern", "waves")

    # Tinta base (V=1). La luminanza la mettiamo con z.
    base_rgb = _hsv_to_rgb01_scalar(hue, sat, 1.0)

    # Griglie una volta sola
    w, h = int(size[0]), int(size[1])
    X = np.linspace(0,1,w, dtype=np.float32)
    Y = np.linspace(0,1,h, dtype=np.float32)
    XX, YY = np.meshgrid(X, Y)
    k = amp * 3.0

    # Buffer riusato per il frame
    frame = np.empty((h, w, 3), dtype=np.uint8)

    def make_frame(t):
        phase = 2*np.pi*(t*speed)

        # z = mappa di luminanza in [0,1]
        if pattern == "waves":
            z = 0.5 + 0.5*np.sin(2*np.pi*(k*XX + 0.7*YY) + phase)

        elif pattern == "ripple":
            r = np.sqrt((XX-0.5)**2 + (YY-0.5)**2)
            z = 0.5 + 0.5*np.sin(30*r - phase*5)

        elif pattern == "electric":
            z = 0.5 + 0.5*np.sin(16*XX + 12*YY + phase*6)

        elif pattern == "snow":
            z = np.full((h, w), val0, dtype=np.float32)
            rng = np.random.default_rng(int(t*40))
            for _ in range(40):
                x = int(rng.integers(0, w)); y = int(rng.integers(0, h))
                y0,y1 = max(0,y-1), min(h,y+1)
                x0,x1 = max(0,x-1), min(w,x+1)
                z[y0:y1, x0:x1] = 1.0

        elif pattern == "rotatingShapes":
            z = np.full((h, w), val0, dtype=np.float32)
            cx, cy = w//2, h//2
            rad = int(min(w,h)/5)
            for i in range(5):
                ang = phase + i*2*np.pi/5
                x = int(cx + rad*np.cos(ang)); y = int(cy + rad*np.sin(ang))
                y0,y1 = max(0,y-2), min(h,y+3)
                x0,x1 = max(0,x-2), min(w,x+3)
                z[y0:y1, x0:x1] = 1.0

        else:
            z = np.full((h, w), 0.5, dtype=np.float32)

        # Applica luminanza e tinta
        z = np.clip(z * val0, 0.0, 1.0).astype(np.float32)
        frame[..., 0] = (z * (base_rgb[0] * 255.0)).astype(np.uint8)
        frame[..., 1] = (z * (base_rgb[1] * 255.0)).astype(np.uint8)
        frame[..., 2] = (z * (base_rgb[2] * 255.0)).astype(np.uint8)
        return frame

    clip = VideoClip(make_frame, duration=duration)
    clip.write_videofile(out_mp4, fps=fps, codec="libx264", audio=False,
                         preset="ultrafast", threads=1, verbose=True)
    return out_mp4

visto che mi sta saltando la ram, proviamo prima a fare la cella per la generazione dell'audio, poi si vede

In [12]:
demo_audio_only = {
    "noise_type":"brown", "duration_sec":6, "lp_cutoff":2500, "hp_cutoff":60,
    "reverb_mix":0.15, "fade_in_sec":0.5, "fade_out_sec":0.5,
    "rain_level":0.0, "waves_level":0.0,
}
_ = render_audio(demo_audio_only, "results/_audio_only.wav")
print("OK audio:", "results/_audio_only.wav")

OK audio: results/_audio_only.wav


ora ci occupiamo della generazione del video muto



In [17]:
params_vis = {
    "pattern":"waves", "duration_sec":6,
    "hue_main":0.55, "sat":0.9, "value":0.7,
    "motion_speed":0.12, "motion_amplitude":0.5,
}
render_video_safe(params_vis, "results/_video_only.mp4", fps=10, size=(320,180))

Moviepy - Building video results/_video_only.mp4.
Moviepy - Writing video results/_video_only.mp4



Moviepy - Done !
Moviepy - video ready results/_video_only.mp4


'results/_video_only.mp4'

SONO ARRIVATA QUI E FINO A QUI RUNNA TUTTO

visto che sincronizzare video e audio mi sfarfalla e mi fa uscire dalla ram visto che è molto pesante, per adesso la soluzione sara quella di creare video e audio indipendentmente (fatto gia nelle celle sopra) e poi di unirle con la funzione merge_audio_video. l'unica cosa sincronizzata sarà la durata del video e la durata dell'audio.

per evitare il crash aggiungiamo questa cosa

In [ ]:
!ffmpeg -y -i results/_video_only_safe.mp4 -i results/_audio_only.wav \
  -c:v copy -c:a aac -b:a 192k -shortest results/_final_av.mp4


In [12]:
from moviepy.editor import VideoFileClip, AudioFileClip

def merge_audio_video(video_path: str, audio_path: str, out_path: str,
                      codec="libx264", audio_codec="aac", preset="ultrafast", threads=1):
    v = VideoFileClip(video_path)
    a = AudioFileClip(audio_path)

    # allinea le durate → taglia audio se più lungo
    a = a.subclip(0, v.duration)

    final = v.set_audio(a)
    final.write_videofile(out_path, fps=v.fps,
                          codec=codec, audio_codec=audio_codec,
                          preset=preset, threads=threads, verbose=True)

    # chiudi risorse
    a.close(); v.close(); final.close()
    return out_path